In [11]:
%pylab inline

import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL seminar").getOrCreate()

Populating the interactive namespace from numpy and matplotlib


/opt/anaconda/envs/bd9/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.types import LongType

schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", StringType()),
    StructField("timestamp", LongType())])
df = spark.read.csv('/labs/laba01/ml-100k/u.data', sep='\t', schema=schema)

In [13]:
df.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [15]:
from pyspark.sql import functions as f
hfilm = df.where(df.item_id == 328).groupby('rating').count().orderBy('rating').select('rating', f.col('count').alias('hist_film'))
hall = df.groupby('rating').count().orderBy('rating').select('rating', f.col('count').alias('hist_all'))
joined_df = hfilm.join(hall, on='rating', how='inner')
hist_film = joined_df.select(joined_df.hist_film).rdd.flatMap(lambda x: x).collect()
hist_all = joined_df.select(joined_df.hist_all).rdd.flatMap(lambda x: x).collect()
joined_df.show()

+------+---------+--------+
|rating|hist_film|hist_all|
+------+---------+--------+
|     1|       12|    6110|
|     2|       40|   11370|
|     3|       94|   27145|
|     4|      109|   34174|
|     5|       40|   21201|
+------+---------+--------+



In [16]:
import json
data = {"hist_film": hist_film,
        "hist_all": hist_all
}
with open('/data/home/farhad.sabilov/lab01.json', 'w') as fout:
    json.dump(data, fout)

In [17]:
spark.stop()